In [1]:
"""
Stack the tm_id and FORTYPCD rasters for the study region
Extract a tm_id lookup table to link to FORTYPCD

Author: maxwell.cook@colorado.edu
"""

import os, sys, time
import pandas as pd
import rioxarray as rxr
import geopandas as gpd
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
from matplotlib.colors import to_rgba

# Custom functions
sys.path.append(os.path.join(os.getcwd(),'code/'))
from __functions import *

proj = 'EPSG:5070'

maindir = '/Users/max/Library/CloudStorage/OneDrive-Personal/mcook/'
projdir = os.path.join(maindir, 'aspen-fire/Aim2/')

print("Ready to go !")

Ready to go !


In [2]:
# load and prepare our study region for cropping TreeMap
# Southern Rockies ecoregion bounds (buffered)
fp = os.path.join(projdir,'data/spatial/raw/boundaries/na_cec_eco_l3_srme.gpkg')
srm = gpd.read_file(fp)
# Crop the raster by the SRM bounds
srm['geometry'] = srm.geometry.buffer(10000)
bounds = srm.total_bounds # total bounds of ecoregion
bounds

array([-1193290.29502988,  1391628.00599962,  -683136.18714099,
        2253336.36986925])

In [3]:
# Load the TreeMap (ca. 2016)
# Pixel values here denote the FIA plot ID ("tm_id")
fp = os.path.join(maindir,'data/landcover/USFS/RDS_TreeMap/TreeMap2016.tif')
treemap_da = rxr.open_rasterio(fp, masked=True, cache=False, chunks='auto').squeeze()
# Load the FORTYPCD raster grid
# Pixel values here represent the FORTYPCD
fp = os.path.join(maindir, 'data/landcover/USFS/RDS_TreeMap/TreeMap2016_FORTYPCD/TreeMap2016_FORTYPCD.tif')
fortypcd_da = rxr.open_rasterio(fp, masked=True, cache=False, chunks='auto').squeeze()

# Grab some raster metadata from one of the layers
shp, gt, wkt, nd = treemap_da.shape, treemap_da.spatial_ref.GeoTransform, treemap_da.rio.crs, treemap_da.rio.nodata
print(
    f"Shape: {shp}; \n"
    f"GeoTransform: {gt}; \n"
    f"WKT: {wkt}; \n"
    f"NoData Value: {nd}; \n"
    f"Data Type: {treemap_da[0].dtype}")

# crop to handle a subset of the data before aligning and stacking
# tm_id raster
treemap_da_c = treemap_da.rio.clip_box(
    minx=bounds[0],
    miny=bounds[1],
    maxx=bounds[2],
    maxy=bounds[3]
)
# FORTYPCD raster
fortypcd_da_c = fortypcd_da.rio.clip_box(
    minx=bounds[0],
    miny=bounds[1],
    maxx=bounds[2],
    maxy=bounds[3]
)

print(f"\nCropped TreeMap data to SRM bounds w/ 10km buffer.")

del treemap_da, fortypcd_da # tidy the CONUS rasters

# Create a data stack with the plot identifier and FORTYPCD
stack_da = xr.Dataset({
    'tm_id': treemap_da_c,
    'fortypcd': fortypcd_da_c
}) # create the data stack
print(f"\n\tCreated the band stack.")
del treemap_da_c, fortypcd_da_c, bounds
gc.collect() # clean up

Shape: (97383, 154221); 
GeoTransform: -2362845.0 30.0 0.0 3180555.0 0.0 -30.0; 
WKT: EPSG:5070; 
NoData Value: nan; 
Data Type: float64

Cropped TreeMap data to SRM bounds w/ 10km buffer.

	Created the band stack.


247

In [4]:
# save the raster stack.
stack_da.rio.write_crs("EPSG:5070", inplace=True)
for band in stack_da.data_vars:
    stack_da[band] = stack_da[band].fillna(-9999)  # Replace NaN with NoData
    stack_da[band].rio.write_nodata(-9999, inplace=True)  # Set NoData value
# Save as a GeoTIFF
out_fp = os.path.join(projdir, "data/spatial/mod/USFS/TreeMap_2016_TMID_FORTYPCD.tif")
stack_da.rio.to_raster(out_fp, compress='zstd', zstd_level=9, dtype='uint16', driver='GTiff')
print(f"Saved multi-band raster to: {out_fp}")
del stack_da
gc.collect()

Saved multi-band raster to: /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/spatial/mod/USFS/TreeMap_2016_TMID_FORTYPCD.tif


3048

In [5]:
# Load the TreeMap (ca. 2016)
# Pixel values here denote the FIA plot ID ("tm_id")
# see '04a_TreeMap_FIA-Prep.ipynb'
# fp = os.path.join(maindir,'data/landcover/USFS/RDS_TreeMap/TreeMap2016.tif') # tm_id band
fp = os.path.join(projdir, "data/spatial/mod/USFS/TreeMap_2016_TMID_FORTYPCD.tif") # multi-band
treemap_da = rxr.open_rasterio(fp, masked=True, cache=False, chunks='auto').squeeze()

# Grab some raster metadata from one of the layers
shp, gt, wkt, nd = treemap_da.shape, treemap_da.spatial_ref.GeoTransform, treemap_da.rio.crs, treemap_da.rio.nodata
print(
    f"Shape: {shp}; \n"
    f"GeoTransform: {gt}; \n"
    f"WKT: {wkt}; \n"
    f"NoData Value: {nd}; \n"
    f"Data Type: {treemap_da[0].dtype}")

# create a mapping linking FIA plot to FORTYPCD (algorithm forest type code)
tmid_vals = treemap_da.sel(band=1).values.flatten()  # tm_id band
fortypcd_vals = treemap_da.sel(band=2).values.flatten()  # FORTYPCD band
pixel_mapping = pd.DataFrame({
    'tm_id': tmid_vals,
    'fortypcd': fortypcd_vals
}).dropna(subset=['tm_id', 'fortypcd'])
pixel_mapping = pixel_mapping.drop_duplicates()
pixel_mapping['tm_id'] = pixel_mapping['tm_id'].astype(int)
pixel_mapping['fortypcd'] = pixel_mapping['fortypcd'].astype(int)
del tmid_vals, fortypcd_vals
gc.collect()
print(pixel_mapping.head(3))

# save this file out.
out_fp = os.path.join(projdir,'data/spatial/mod/USFS/treemap_tmid_fortypcd_lookup.csv')
pixel_mapping.to_csv(out_fp)
print(f"Saved file to: {out_fp}")

Shape: (2, 28724, 17006); 
GeoTransform: -1193295.0 30.0 0.0 2253345.0 0.0 -30.0; 
WKT: EPSG:5070; 
NoData Value: nan; 
Data Type: float32
   tm_id  fortypcd
0  20038       268
1  20976       266
2  37567       367
Saved file to: /Users/max/Library/CloudStorage/OneDrive-Personal/mcook/aspen-fire/Aim2/data/spatial/mod/USFS/treemap_tmid_fortypcd_lookup.csv


In [ ]:
# create a forest/non-forest grid
forest_mask = xr.where(~np.isnan(tmid_da), 1, 0)

# Save as a GeoTIFF
out_fp = os.path.join(projdir, "data/spatial/mod/USFS/TreeMap_2016_forestmask.tif")
forest_mask.rio.to_raster(out_fp, compress='zstd', zstd_level=9, dtype='uint16', driver='GTiff')
print(f"Saved multi-band raster to: {out_fp}")

# calculate the total grid pixels
# see __functions.py 'compute_band_stats'
forest_pct = compute_band_stats(grid, forest_mask, 'grid_index', attr='constant')
forest_pct.rename(columns={
    'pct_cover': 'forest_pct',
    'total_pixels': 'forest_pixels'
}, inplace=True)
forest_pct = forest_pct[forest_pct['constant'] == 1] # just forest pixels
forest_pct = forest_pct[['grid_index','forest_pixels','forest_pct']]

# how many forested grids are there? (>50%)
n_forest = len(forest_pct[forest_pct['forest_pct'] > 50])
print(f"\n[{round(n_forest/len(grid)*100,2)}%] predominantly forest grids.")
print(f"\n{forest_pct.head(3)}")

del forest_mask
gc.collect() # clean up